In [1]:
from pathlib import Path
import os

yr = 2024
d = 17

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [2]:
import copy
import re

def format_input(inp):

  registers = {'A': int(re.sub('[^0-9]','', inp[0])),
               'B': int(re.sub('[^0-9]','', inp[1])),
               'C': int(re.sub('[^0-9]','', inp[2]))}
  
  program = [int(x) for x in re.sub('[^0-9,]','', inp[4]).split(',')]
  return registers, program

In [3]:
def adv(opcode, operand, combo, pointer, registers, out):
  '''0'''
  assert(opcode==0)
  registers['A'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out

def bxl(opcode, operand, combo, pointer, registers, out):
  '''1'''
  assert(opcode==1)
  registers['B'] = registers['B'] ^ operand
  pointer += 2
  return pointer, registers, out

def bst(opcode, operand, combo, pointer, registers, out):
  '''2'''
  assert(opcode==2)
  registers['B'] = combo%8
  pointer += 2
  return pointer, registers, out

def jnz(opcode, operand, combo, pointer, registers, out):
  '''3'''
  assert(opcode==3)
  if registers['A']!=0:
    pointer = operand-2
  pointer += 2
  return pointer, registers, out

def bxc(opcode, operand, combo, pointer, registers, out):
  '''4'''
  assert(opcode==4)
  registers['B'] = registers['B'] ^ registers['C']
  pointer += 2
  return pointer, registers, out

def out(opcode, operand, combo, pointer, registers, out):
  '''5'''
  assert(opcode==5)
  out.append(combo%8)
  pointer += 2
  return pointer, registers, out


def bdv(opcode, operand, combo, pointer, registers, out):
  '''6'''
  assert(opcode==6)
  registers['B'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out


def cdv(opcode, operand, combo, pointer, registers, out):
  '''7'''
  assert(opcode==7)
  registers['C'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out

  
opcode2func = {0:adv,
               1:bxl,
               2:bst,
               3:jnz,
               4:bxc,
               5:out,
               6:bdv,
               7:cdv}


def execute(registers, program):
  pointer = 0
  out = []
  while pointer < len(program):
    operand_to_combo = {
                      0:0,
                      1:1,
                      2:2,
                      3:3,
                      4: registers['A'],
                      5: registers['B'],
                      6: registers['C']
                      }
    opcode = program[pointer]
    operand = program[pointer+1]
    combo = operand_to_combo[operand]
    pointer, registers, out = opcode2func[opcode](opcode, operand, combo, pointer, registers, out)


  return out 


def get_output(registers, program):
  return ','.join(map(str,execute(registers, program)))


def calculate_A_register(registers, program):
  def calculate_A_register_(registers, program, cur_num="", num_to_check=1):
    out = []
    binary_0_to_8 = ["{:b}".format(i).zfill(3) for i in range(0,8)]
    potential_solutions = []
    for b in binary_0_to_8:
      cur_registers = copy.deepcopy(registers)
      cur_program = copy.deepcopy(program)
      cur_registers['A'] = int(cur_num+b, 2)
      out = execute(cur_registers, cur_program)
      if out == program[-num_to_check:]:
          if out == program:
            potential_solutions.append(cur_num+b)
          else:
            potential_solutions.append(calculate_A_register_(registers, program, cur_num + b, num_to_check+1))
    out_num = b if len(potential_solutions) == 0 else max(potential_solutions, key=len)
    return out_num
  return int(calculate_A_register_(registers, program), 2)

In [4]:
import time

t = time.time()

formatted_input = format_input(inp)

print(get_output(*copy.deepcopy(formatted_input)))
print(calculate_A_register(*formatted_input))

print('\nRUNTIME: ', time.time()-t)

3,6,7,0,5,7,3,1,4
164278496489149

RUNTIME:  0.052858591079711914
